In [1]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import h5py

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as F

import lava.lib.dl.slayer as slayer

from gsc_dataset import GSCDataset

import numpy as np

In [2]:
torch.manual_seed(4205)

# Event sparsity loss

Sparsity loss to penalize the network for high event-rate.

In [3]:
def event_rate_loss(x, max_rate=0.01):
    mean_event_rate = torch.mean(torch.abs(x))
    return F.mse_loss(F.relu(mean_event_rate - max_rate), torch.zeros_like(mean_event_rate))

def loss(output, target):
    return F.mse_loss(output, target.to("cpu")) #TODO: CUDA

# Network description

__SLAYER 2.0__ (__`lava.dl.slayer`__) provides a variety of learnable _neuron models_ <!-- (`slayer.neuron.{cuba, rf, ad_lif, __sigma_delta__, ...}`)  --> , _synapses_ <!-- (`slayer.{synapse, complex.synapse}.{dense, conv, pool, convT, unpool}`)  --> _axons_ and _dendrites_ that support quantized training. 
For easier use, it also provides __`block`__ interface which packages the associated neurons, synapses, axons and dendrite features into a single module. 

__Sigma-delta blocks__ are available as `slayer.blocks.sigma_delta.{Dense, Conv, Pool, Input, Output, Flatten, ...}` which can be easily composed to create a variety of sequential network descriptions as shown below. The blocks can easily enable _synaptic weight normalization_, _neuron normalization_ as well as provide useful _gradient monitoring_ utility and _hdf5 network export_ utility.

<!-- TODO:
- Describe how easy it is to describe a network in slayer2.0
- Parameter Quantization is automatically handled unless disabled
- Weight and neuron normalization
- gradient monitoring utility
- hdf5 export utility -->

These blocks can be used to create a network using standard PyTorch procedure.

In [4]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        cuba_params = {
                'threshold'    : 1.25, 
                'current_decay': 0.25, 
                'voltage_decay': 0.25, 
                'tau_grad'     : 0.1,
                'scale_grad'   : 0.8,
                'shared_param' : False, 
                'requires_grad': False, 
                'graded_spike' : False,
            }

        recurr_weight_scale = 1.0

        self.blocks = torch.nn.ModuleList([
                # layer 1
                slayer.block.cuba.Dense(cuba_params, 80, 512, weight_scale=recurr_weight_scale),
                slayer.block.cuba.Dense(cuba_params, 512, 35, weight_scale=1.0)
                #slayer.block.cuba.Average(num_outputs=35) #TODO 35
            ])

        
    def forward(self, spike):
        count = []

        for block in self.blocks:
            # print(block)
            # print(f'{block=}')
            spike = block(spike)
            # print("spike computed")
            if self.count_calc:
                count.append(torch.mean(spike).item())
                if np.isnan(count[-1]):
                    print(spike)
                    return spike, None

        return (
            torch.mean(spike, dim=-1), 
            torch.FloatTensor(count).reshape((1, -1)).to(spike.get_device()) if self.count_calc else None,)
    

    def grad_flow(self, path):
        # helps monitor the gradient flow
        grad = [b.synapse.grad_norm for b in self.blocks if hasattr(b, 'synapse')]

        plt.figure()
        plt.semilogy(grad)
        plt.savefig(path + 'gradFlow.png')
        plt.close()

        return grad
    
    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))
        
            

# Training parameters

In [5]:
batch  = 8  # batch size
lr     = 0.001 # leaerning rate
lam    = 0.01  # lagrangian for event rate loss
epochs = 20  # training epochs
steps  = [60, 120, 160] # learning rate reduction milestones

trained_folder = 'Trained'
logs_folder = 'Logs'

os.makedirs(trained_folder, exist_ok=True)
os.makedirs(logs_folder   , exist_ok=True)

device = torch.device('cpu')

In [6]:
# Datasets
training_set = GSCDataset(
    train=True, 
    transform=transforms.Compose([
        transforms.ToTensor(),
    ]), 
)

Dataset not available locally. Starting download ...
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Ue4XohCOV5YXy57S_5tDfCVqzLr101M7' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Ue4XohCOV5YXy57S_5tDfCVqzLr101M7" -O data/driving_dataset.zip && rm -rf /tmp/cookies.txt


--2024-04-19 06:22:44--  https://docs.google.com/uc?export=download&confirm=&id=1Ue4XohCOV5YXy57S_5tDfCVqzLr101M7
Resolving docs.google.com (docs.google.com)... 142.251.211.238, 2607:f8b0:400a:804::200e
Connecting to docs.google.com (docs.google.com)|142.251.211.238|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1Ue4XohCOV5YXy57S_5tDfCVqzLr101M7&export=download [following]
--2024-04-19 06:22:44--  https://drive.usercontent.google.com/download?id=1Ue4XohCOV5YXy57S_5tDfCVqzLr101M7&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.217.97, 2607:f8b0:400a:80b::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.217.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2431 (2.4K) [text/html]
Saving to: ‘data/driving_dataset.zip’

     0K ..                                                    100% 65.6M=0s

Download complete.
Extracting data (this may take a while) ...
Could not extract file data/driving_dataset.zip. Please extract it manually.


FileNotFoundError: [Errno 2] No such file or directory: 'data/driving_dataset/data.txt'

# Instantiate Network, Optimizer, Dataset and Dataloader

In [ ]:
net = Network().to(device)

optimizer = torch.optim.RAdam(net.parameters(), lr=lr, weight_decay=1e-5)

# Datasets
training_set = GSCDataset(
    train=True, 
    transform=transforms.Compose([
        transforms.ToTensor(),
    ]), 
)
testing_set = GSCDataset(
    train=False, 
    transform=transforms.Compose([
        transforms.ToTensor(),
    ]),
)

train_loader = DataLoader(dataset=training_set, batch_size=batch, shuffle=True, num_workers=8)
test_loader  = DataLoader(dataset=testing_set , batch_size=batch, shuffle=True, num_workers=8)

stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(
        net=net,
        error=lambda output, target: F.mse_loss(output.flatten(), target.flatten()),
        optimizer=optimizer,
        stats=stats,
        count_log=True,
        lam=lam
    )

In [ ]:
# Load dataset
x_train = np.load(os.path.expanduser("/homes/ts468/data/rawSC/rawSC_80input/") + "training_x_data.npy")
y_train = np.load(os.path.expanduser("/homes/ts468/data/rawSC/rawSC_80input/") + "training_y_data.npy")

x_test = np.load(os.path.expanduser("/homes/ts468/data/rawSC/rawSC_80input/") + "testing_x_data.npy")
y_test = np.load(os.path.expanduser("/homes/ts468/data/rawSC/rawSC_80input/") + "testing_y_data.npy")

training_images = x_train #np.swapaxes(x_train, 1, 2) 
testing_images = x_test #np.swapaxes(x_test, 1, 2) 

training_images = training_images + abs(np.floor(training_images.min()))
testing_images = testing_images + abs(np.floor(testing_images.min()))

training_labels = y_train
testing_labels = y_test

# adding validation data if exists
validation_images = np.array([])
validation_labels = np.array([])
if os.path.isfile(os.path.expanduser("/homes/ts468/data/rawSC/rawSC_80input/") + "validation_y_data.npy"):
        print("!! validation dataset loaded successfully")
        x_validation = np.load(os.path.expanduser("/homes/ts468/data/rawSC/rawSC_80input/") + "validation_x_data.npy")
        y_validation = np.load(os.path.expanduser("/homes/ts468/data/rawSC/rawSC_80input/") + "validation_y_data.npy")

        validation_images = x_validation #np.swapaxes(x_validation, 1, 2) 
        validation_images = validation_images + abs(np.floor(validation_images.min()))

        validation_labels = y_validation

training_images = np.expand_dims(training_images, 1)
testing_images = np.expand_dims(testing_images, 1)
validation_images = np.expand_dims(validation_images, 1)

In [ ]:
train_loader = DataLoader(dataset=training_set, batch_size=batch, shuffle=True, num_workers=8)
test_loader  = DataLoader(dataset=testing_set , batch_size=batch, shuffle=True, num_workers=8)

# Training loop

Training loop mainly consists of looping over epochs and calling `assistant.train` and `assistant.test` utilities over training and testing dataset. The `assistant` utility takes care of statndard backpropagation procedure internally.

* `stats` can be used in print statement to get formatted stats printout.
* `stats.testing.best_loss` can be used to find out if the current iteration has the best testing loss. Here, we use it to save the best model.
* `stats.update()` updates the stats collected for the epoch.
* `stats.save` saves the stats in files.

In [ ]:
for epoch in range(epochs):
    if epoch in steps:
        for param_group in optimizer.param_groups:    
            print('\nLearning rate reduction from', param_group['lr'])
            param_group['lr'] /= 10/3
        
    for i, (input, ground_truth) in enumerate(train_loader): # training loop
        assistant.train(input, ground_truth)
        print(f'\r[Epoch {epoch:3d}/{epochs}] {stats}', end='')
    
    for i, (input, ground_truth) in enumerate(test_loader): # testing loop
        assistant.test(input, ground_truth)
        print(f'\r[Epoch {epoch:3d}/{epochs}] {stats}', end='')
        
    if epoch%50==49: print() 
    if stats.testing.best_loss:  
        torch.save(net.state_dict(), trained_folder + '/network.pt')
    stats.update()
    stats.save(trained_folder + '/')
    
    # gradient flow monitoring
    net.grad_flow(trained_folder + '/')
    
    # checkpoint saves
    if epoch%10 == 0:
        torch.save({'net': net.state_dict(), 'optimizer': optimizer.state_dict()}, logs_folder + f'/checkpoint{epoch}.pt')                   

# Learning plots.

Plotting the learning curves is as easy as calling `stats.plot()`.

In [ ]:
stats.plot(figsize=(15, 5))

# Export the best trained model

Load the best model during training and export it as hdf5 network. It is supported by `lava.lib.dl.netx` to automatically load the network as a lava process.

In [ ]:
net.load_state_dict(torch.load(trained_folder + '/network.pt'))
net.export_hdf5(trained_folder + '/network.net')

# Operation count of trained model

Here, we compare the synaptic operation and neuron activity of the trained SDNN and an ANN of iso-architecture.

## Event statistics on testing dataset

In [ ]:
counts = []
for i, (input, ground_truth) in enumerate(test_loader):
    _, count = assistant.test(input, ground_truth)
    count = (count.flatten()/(input.shape[-1]-1)/input.shape[0]).tolist() # count skips first events
    counts.append(count) 
    print('\rEvent count : ' + ', '.join([f'{c:.4f}' for c in count]), f'| {stats.testing}', end='') 
        
counts = np.mean(counts, axis=0)

# Event and Synops comparion with ANN

In [ ]:
utils.compare_ops(net, counts, mse=stats.testing.min_loss)

### How to learn more?

If you want to learn more about Sigma-Delta neurons, take a look at the [Lava tutorial](https://github.com/lava-nc/lava/blob/main/tutorials/in_depth/tutorial10_sigma_delta_neurons.ipynb).

Find out more about Lava and have a look at the [Lava documentation](https://lava-nc.org/ "Lava Documentation") or dive into the [source code](https://github.com/lava-nc/lava/ "Lava Source Code").

To receive regular updates on the latest developments and releases of the Lava Software Framework please subscribe to the [INRC newsletter](http://eepurl.com/hJCyhb "INRC Newsletter").